<a href="https://colab.research.google.com/github/Maverick-Ansh/Karpathy-_neural_networks/blob/main/Karpathy__Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>